In [1]:
import re
import jieba
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.metrics import classification_report
import warnings

warnings.filterwarnings("ignore")

In [2]:
def generate_dataframe(filepath):
    """
    generate a dataframe to store the comments and labels
    """
    _comments = (
        open(filepath, encoding="utf-8-sig").read().replace("\n", "").split("</review>")
    )
    # remove empty elements
    # 移除空元素
    _comments = list(filter(None, _comments))
    # while "" in _comments:
    #     _comments.remove("")
    # create DataFrame object to store Comments and Labels
    local_comments = []
    local_labels = []
    if filepath.find("pos") > 0:  # for pos file
        for i in range(len(_comments)):
            pattern = re.compile("^<.*>(.*)")
            text = pattern.findall(_comments[i])
            local_comments.append(text[0])
        df = pd.DataFrame(local_comments, columns=["Comment"])
        label_pos = [1 for local_comments in range(len(local_comments))]
        df["Label"] = label_pos
    elif filepath.find("neg") > 8:  # for neg file
        for j in range(len(_comments)):
            # TODO 2：编译正则表达式，生成一个字符串为'^<.*>(.*)'的正则表达式对象pattern
            pattern = re.compile("^<.*>(.*)")
            # TODO 3：在字符串＿comments[j］中找到正则表达式pattern所匹配的所有子串，并返回一个列表
            text = pattern.findall(_comments[j])
            local_comments.append(text[0])
        # TODO 4: 通过local_comments和columns=['Comment'］参数生成一个DataFrame对象df
        df = pd.DataFrame(local_comments, columns=["Comment"])
        # TODO 5：生成一个label_neg列表存储train_negative.txt中负面评论的标签，列表长度为local_comments
        label_neg = [0 for local_comments in range(len(local_comments))]
        # TODO 6：将label_neg增加为对象df的'Label'列
        df["Label"] = label_neg
    else:  # for combined file
        for k in range(len(_comments)):
            pattern = re.compile("^<.*>(.*)")
            text = pattern.findall(_comments[k])
            local_comments.append(text[0])
            pattern = re.compile("<.*>")
            numbers = pattern.findall(_comments[k])
            local_labels.append(numbers[0][-3])
            local_labels = [int(x) for x in local_labels]
        df = pd.DataFrame(local_comments, columns=["Comment"])
        df["Label"] = local_labels
    return df

In [3]:
def process_file():
    """
    读取训练数据和测试数据，并对它们做一些预处理
    """
    global train_comments
    global train_labels
    global test_comments
    global test_labels
    train_pos_file = "/CODE/Python/Machine_learning/23.11.1/train_positive.txt"
    train_neg_file = "/CODE/Python/Machine_learning/23.11.1/train_negative.txt"
    test_comb_file = "/CODE/Python/Machine_learning/23.11.1/test_combined.txt"
    df_pos = generate_dataframe(train_pos_file)
    df_neg = generate_dataframe(train_neg_file)
    df_com = generate_dataframe(test_comb_file)
    train_comments = list(df_pos.append(df_neg)["Comment"])
    # TODO 1：把具体的内容写入到变量里面
    train_labels = list(df_pos.append(df_neg)["Label"])
    test_comments = list(df_com["Comment"])
    test_labels = list(df_com["Label"])

In [4]:
train_comments = []
train_labels = []
test_comments = []
test_labels = []

process_file()

print("****************Train Comments****************:\n", train_comments)
print("****************Train Labels****************:\n", train_labels)
print("****************TestComments****************:\n", test_comments)
print("****************TestLabels****************:\n", test_labels)

****************Train Comments****************:
 ['请问这机不是有个遥控器的吗？', '发短信特别不方便！背后的屏幕很大用起来不舒服，是手触屏的！切换屏幕很麻烦！', '手感超好，而且黑色相比白色在转得时候不容易眼花，找童年的记忆啦。', '！！！！！', '先付款的   有信用', '价格质量售后都很满意', '书的质量和印刷都不错，字的大小也刚刚好，很清楚，喜欢', '超级值得看的一个电影', '今天突然看到卓越有卖这个的，可是韩国不是卖没有了吗。虽然是引进版的，可是之前也卖没有了。卓越从哪里找出来的啊', '江大，继续写', '还不错，得多跟着练才能跟的上~~', '我前天两本一起买还是42.8元的，怎么一下子就降价了。。。。但书还是对我有很大帮助的。而且发货速度很快，隔天就到了~！', '内容自不必细说，80后的人都知道，质量比较好，虽然不是尽善尽美，但企事业算是不错了。', '不是的话，看了这本书后我相信那就是真有时间隧道。', '发货迅速，书的质量也很好~很满意~', '作为一本指导性质的编程类书籍，这本书将各种设计模式演绎的淋漓尽致，通俗易懂，看后使人获益匪浅！设计模式只是开始，要真正的会用、用好还是需要实际的磨练。', '绝对不容错过的收藏！', '出售和购买盗版都是违法行为，你们还敢随便在这里瞎起哄。自己想想自己像是什么人吧', '不知道拨号数字大不大，话筒音量大不大。。。。', '神秘园的曲子总是能打动每一个人心中最真的地方，太好听了~', '上次买五月天的专辑和几本书，我特别说明了要包装牢固，joyo用了一个箱子，我很感动。这次也是cd和几本书，也是有包装的特别说明，结果竟然只有胶带，气死我了！还好，没有破损，要不然对不起布兰妮啊！joyo的服务怎么每况愈下呢？', '挺好！有空我也去听听！', '鸟哥就是鸟哥。书对我这linux的初学者来说很有帮助。可惜 少有时间看', '发货很即时，很快', '第一眼看到这款手机太喜欢了,而且价格也不贵.朋友有很多都买了新手机,我也想买,但现在我的手机还好好的,真的很想要.没办法,继续关注吧!!!!!!!!!!!', '非常不错的一部歌舞电影！很青春！', '四天左右到的货，书很整洁', '用起来很不错，没有出现问题，价格也不贵', '挺不错。老狼还是没有什

In [5]:
def clean_symbols(text):
    # 对特殊符号做一些处理，比如把多个感叹号变成一个，去掉空格等
    text = re.sub("[!! ]+", "!", text)
    text = re.sub("[?? ]+", "?", text)
    text = re.sub("[a-zA-Z#$%&'()*+,-./:;：<=>@，。★、…【】《》＂“”‘’[\\]^_`{|}~]+", "OOV", text)
    return re.sub("\s+", " ", text)

In [6]:
stopwords_path = "/CODE/Python/Machine_learning/23.11.8/stopwords.txt"
stopwords = open(stopwords_path, encoding="utf-8").read().splitlines()
# 1．去掉特殊符号
# 2．把数字转换成特殊字符或者单词
# 3．分词并做停用词过滤
train_comments_cleaned = []
for train_comment in train_comments:
    # TODO 1．调用自定义的clean_symbols数理io特殊
    train_comment = clean_symbols(train_comment)
    # TODO 2．调用正则表达式的re.sub函数把train_comment中的数字转换成特殊单词' NUM '
    train_comment = re.sub("\d", " NUM ", train_comment)
    # TODO 3．调用jieba.cut函数对train_comment进行分词，cut_all参数为默认值
    cut_train = jieba.cut(train_comment)
    # 实现训练数据的停用词过滤
    seg_train = [word for word in cut_train if word not in stopwords]
    train_comment_cleaned = " ".join(seg_train)
    train_comments_cleaned.append(train_comment_cleaned)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LLeavee\AppData\Local\Temp\jieba.cache
Loading model cost 0.795 seconds.
Prefix dict has been built successfully.


In [7]:
test_comments_cleaned = []
for test_comment in test_comments:
    # TODO 4．调用自定义的clean_symbols函数处理test_comment中的特殊符号
    test_comment = clean_symbols(test_comment)
    # TODO 5．调用正则表达式的re.sub函数把test_comment中的数字转换成特殊单词'NUM'
    test_comment = re.sub("\d", " NUM ", test_comment)
    # TODO 6．调用jieba.cut函数对train_comment进行分词，cut_all参数为默认值
    cut_test = jieba.cut(test_comment)
    # 实现测试数据的停用词过滤
    seg_test = [word for word in cut_test if word not in stopwords]
    test_comment_cleaned = "".join(seg_test)
    test_comments_cleaned.append(test_comment_cleaned)
# 分别打印前5行
for i in range(0, 5):
    print(train_comments_cleaned[i])
for i in range(0, 5):
    print(test_comments_cleaned[i])
# TODO 7：利用tf-idf从文本中提取特征，写到数组里面。
# 参考：https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
# 训练数据的特征，使用fit_transform函数
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_comments_cleaned)
# 训练数据的Label
y_train = train_labels
# 测试数据的特征，使用transform函数
X_test = vectorizer.transform(test_comments_cleaned)
# 测试数据的Label
y_test = test_labels
print(np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test))

请问 这机 不是 有个 遥控器
发短信 特别 不 方便 背后 屏幕 很大 起来 不 舒服 OOV 手触 屏 切换 屏幕 很 麻烦
手感 超好 OOV 黑色 相比 白色 转得 不 容易 眼花 OOV 找 童年 记忆 OOV

先 付款 ! 信用
终于找到同道中人初中OOV已经喜欢上OOV同学都鄙夷眼光看OOV人为OOV样子古怪说OOV丑OOV当场气晕现在同道中人OOV好开心OOV!OOV
看完已深夜两点OOV却坐在电脑前情难自禁OOV这是最好结局OOV惟有OOV就让前世今生纠结停留此刻OOV再相逢时OOV愿人生不再人唏嘘OOV身心会只居一处OOV痛心人OOV爱OOV
袁阔成先生当今评书界泰斗OOV十二金钱镖代表作
确实非常不错OOV物有所值
基本上说诈骗
(8065, 23239) (2500, 23239) (8065,) (2500,)


In [8]:
# # 利用逻辑回归来训练模型
# params_c = np.logspace(-5, 2, 15)
# model = GridSearchCV(
#     estimator=LogisticRegression(penalty="l2"),
#     param_grid={"C": params_c},
#     scoring="f1",
#     cv=10,
# )
# model.fit(X_train, y_train)
# predictions = model.predict(X_test)
# print(classification_report(y_test, predictions))

# # 利用朴素贝叶斯来训练模型
# params_alpha = [1]
# model = GridSearchCV(
#     estimator=MultinomialNB(fit_prior=True),
#     param_grid={"alpha": params_alpha},
#     scoring="f1",
#     cv=10,
# )
# model.fit(X_train, y_train)
# predictions = model.predict(X_test)
# print(classification_report(y_test, predictions))

# 利用SVM来训练模型
params = [
    {"kernel": ["linear"], "C": [1, 10]},
    {"kernel": ["poly"], "C": [1]},
    {"kernel": ["rbf"], "C": [10], "gamma": [1, 0.1]},
]
model = GridSearchCV(estimator=svm.SVC(), param_grid=params, scoring="f1", cv=10)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))